In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) 

In [5]:
bills = pd.read_csv('data_store/dataframes/full_congress_level_df8.csv')

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3,4,7,8,9,12,16,17,18,20,21,24,25,26,29,37,38,39,42,45,46,47,50,53,54,55,58,61,62,63,66,69,70,71,74,77,78,79,82,85,86,87,90,93,94,95,98,101,102,103,106,109,110,111,114,115,116,117,144,145,146,150,151,152,154,155,158,159,160,163,166,167,168,170,171,174,175,176,178,179,182,183,184,187,194,195,196,200,201,202,206,208,211,212,213,215,216,221,222,223,226,229,230,231,234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
bills = bills[bills['congress'] > 92].reset_index(drop=True) 

In [7]:
len(bills)

255449

In [2]:
def vote_dataframe_builder(first_congress, last_congress):
    vote_files = []
    congress_numbers = range(first_congress, last_congress+1)
    for congress in congress_numbers:
        vote_files.append('data_store/votes/congress_'+str(congress)+'_member_votes.csv')
    file_list = []
    for filename in vote_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        file_list.append(df)
    dataframe = pd.concat(file_list, axis=0, ignore_index=True)
    return dataframe 

In [6]:
member_df = pd.read_csv('all_legislators.csv')

In [12]:
member_df['party'].unique()

array(['Democrat', 'Independent', 'Republican', 'Anti-Administration',
       nan, 'Pro-Administration', 'Federalist', 'Democratic Republican',
       'Unknown', 'Adams', 'Jackson', 'Jackson Republican',
       'Crawford Republican', 'Whig', 'Anti-Jacksonian', 'Adams Democrat',
       'Nullifier', 'Anti Masonic', 'Anti Jacksonian', 'Jacksonian',
       'Anti Jackson', 'Union Democrat', 'Conservative', 'Ind. Democrat',
       'Law and Order', 'American', 'Liberty', 'Free Soil',
       'Ind. Republican-Democrat', 'Ind. Whig', 'Unionist',
       'States Rights', 'Anti-Lecompton Democrat',
       'Constitutional Unionist', 'Independent Democrat',
       'Unconditional Unionist', 'Conservative Republican',
       'Ind. Republican', 'Liberal Republican', 'National Greenbacker',
       'Readjuster Democrat', 'Readjuster', 'Union', 'Union Labor',
       'Populist', 'Silver Republican', 'Free Silver',
       'Democratic and Union Labor', 'Progressive Republican',
       'Progressive', 'Prohibit

In [3]:
vote_df = vote_dataframe_builder(93, 116) 

In [4]:
vote_df['bill_id'].nunique() 

7382

In [29]:
vote_df['vote_id'].nunique()

28057

In [10]:
testing_ids = vote_df[vote_df['vote_id'] == 'hr2107-93-8'].reset_index(drop=True) 

In [18]:
testing_ids['bill_id'][0]

'hr2107-93'

In [19]:
def party_positions(vote_df):
    
    yea_vote_df = vote_df[vote_df['vote'] == 1].reset_index(drop=True) 
    nay_vote_df = vote_df[vote_df['vote'] == -1].reset_index(drop=True)

    r_yeas = []
    d_yeas = []
    r_nays = []
    d_nays = []
    bill_id = []
    vote_id = [] 
    print ('Compiling vote records by party...')
    print ()
    unique_votes = vote_df['vote_id'].unique()
    
    for vote in unique_votes:
        vote_id.append(vote)
        sub_yea = yea_vote_df[yea_vote_df['vote_id'] == vote].reset_index(drop=True)
        sub_nay = nay_vote_df[nay_vote_df['vote_id'] == vote].reset_index(drop=True)
        if len(sub_yea) > 0:
            bill_id.append(sub_yea['bill_id'][0])
        else:
            bill_id.append(sub_nay['bill_id'][0])
        r_yeas.append(len(sub_yea[sub_yea['party'] == 'Republican']))
        d_yeas.append(len(sub_yea[sub_yea['party'] == 'Democrat'])) 
        r_nays.append(len(sub_nay[sub_nay['party'] == 'Republican']))
        d_nays.append(len(sub_nay[sub_nay['party'] == 'Democrat'])) 
    
    output_dict = {'vote_id': vote_id, 'bill_id': bill_id,
                   'rep_yeas': r_yeas, 'dem_yeas': d_yeas,
                   'rep_nays': r_nays, 'dem_nays': d_nays}
    party_position_df = pd.DataFrame(output_dict) 
    party_position_df['rep_support'] = (
        party_position_df['rep_yeas'] / (
            party_position_df['rep_yeas'] + party_position_df['rep_nays']))
    party_position_df['dem_support'] = (
        party_position_df['dem_yeas'] / (
            party_position_df['dem_yeas'] + party_position_df['dem_nays']))
    
    majority_position = []
    print ('Determining party majority positions...')
    print ()
    for vote in range(len(party_position_df)):
        if party_position_df['rep_support'][
            vote] > 0.95 and party_position_df['dem_support'][vote] > 0.95:
            majority_position.append('B')
        elif party_position_df['rep_support'][
            vote] > party_position_df['dem_support'][vote]:
            majority_position.append('R')
        else:
            majority_position.append('D')
    
    party_position_df['majority_party'] = majority_position
    
    
    return party_position_df         

In [20]:
party_positions = party_positions(vote_df) 

Compiling vote records by party...

Determining party majority positions...



In [22]:
test_str = 'hr2107-93'

In [23]:
print(test_str.rpartition('-')[-1])

93


In [33]:
party_positions.head() 

,vote_id,bill_id,rep_yeas,dem_yeas,rep_nays,dem_nays,rep_support,dem_support,majority_party,congress
0,hr2107-93-8,hr2107-93,150,25,23,192,0.867052,0.115207,R,93
1,hr2107-93-9,hr2107-93,119,13,54,203,0.687861,0.060185,R,93
2,hr2107-93-10,hr2107-93,54,197,124,20,0.303371,0.907834,D,93
3,hr3694-93-11,hr3694-93,127,157,31,41,0.803797,0.792929,R,93
4,hr1975-93-13,hr1975-93,17,176,138,20,0.109677,0.897959,D,93


In [27]:
congress_numbers = []
for row in range(len(party_positions)):
    congress_numbers.append(party_positions['bill_id'][row].rpartition('-')[-1])  

In [32]:
party_positions['congress'] = congress_numbers

In [34]:
party_positions.to_csv('party_position_df.csv', index=False) 

In [8]:
party_positions = pd.read_csv('party_position_df.csv') 

In [35]:
bill_topics = bills[['bill_id', 'major_label 1', 'major_label 2', 'major_label 3']] 

In [47]:
df = pd.merge(left=party_positions, right=bill_topics, how='left', on='bill_id') 

In [49]:
len(df) 

28057

In [39]:
len(party_positions) 

28057

In [50]:
df.head()

,vote_id,bill_id,rep_yeas,dem_yeas,rep_nays,dem_nays,rep_support,dem_support,majority_party,congress,major_label 1,major_label 2,major_label 3
0,hr2107-93-8,hr2107-93,150,25,23,192,0.867052,0.115207,R,93,7.0,14.0,20.0
1,hr2107-93-9,hr2107-93,119,13,54,203,0.687861,0.060185,R,93,7.0,14.0,20.0
2,hr2107-93-10,hr2107-93,54,197,124,20,0.303371,0.907834,D,93,7.0,14.0,20.0
3,hr3694-93-11,hr3694-93,127,157,31,41,0.803797,0.792929,R,93,20.0,21.0,12.0
4,hr1975-93-13,hr1975-93,17,176,138,20,0.109677,0.897959,D,93,4.0,15.0,20.0


In [ ]:
votes_and_parties_df.to_csv('new_vote_and_parties_df.csv')

In [53]:
votes_and_parties_df = pd.merge(left=vote_df, right=df, how='left', on='vote_id') 

In [2]:
votes_and_parties_df = pd.read_csv('new_vote_and_parties_df.csv', index_col=0)

/home/william/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
votes_and_parties_df.dropna(subset=['major_label 1'], inplace=True)
votes_and_parties_df.dropna(subset=['major_label 2'], inplace=True)
votes_and_parties_df.dropna(subset=['major_label 3'], inplace=True) 
votes_and_parties_df['bill_num'] = votes_and_parties_df[
    'bill_number'].str.replace('^[^\d]*', '').astype(int)
votes_and_parties_df['bill_type'] = votes_and_parties_df[
    'bill_number'].str.replace('[^a-zA-Z]', '')
votes_and_parties_df.sort_values(['congress', 'bill_type', 'bill_num', 'rollnumber'],
                                 inplace=True)
votes_and_parties_df.reset_index(drop=True, inplace=True)

In [4]:
votes_and_parties_df.head(5)

,chamber,rollnumber,date,yea_count,nay_count,bill_number,bioguide_id,thomas_id,party,vote_id,vote,rep_yeas,dem_yeas,rep_nays,dem_nays,rep_support,dem_support,majority_party,major_label 1,major_label 2,major_label 3,congress,bill_id,bill_num,bill_type
0,House,575,1974-02-28,179.0,218.0,HR2,A000216,27,Republican,hr2-93-575,1,138,40,38,177,0.784091,0.184332,R,5.0,20.0,15.0,93,hr2-93,2,HR
1,House,575,1974-02-28,179.0,218.0,HR2,A000222,32,Democrat,hr2-93-575,-1,138,40,38,177,0.784091,0.184332,R,5.0,20.0,15.0,93,hr2-93,2,HR
2,House,575,1974-02-28,179.0,218.0,HR2,B000178,59,Democrat,hr2-93-575,-1,138,40,38,177,0.784091,0.184332,R,5.0,20.0,15.0,93,hr2-93,2,HR
3,House,575,1974-02-28,179.0,218.0,HR2,B000371,77,Democrat,hr2-93-575,-1,138,40,38,177,0.784091,0.184332,R,5.0,20.0,15.0,93,hr2-93,2,HR
4,House,575,1974-02-28,179.0,218.0,HR2,B000550,95,Democrat,hr2-93-575,-1,138,40,38,177,0.784091,0.184332,R,5.0,20.0,15.0,93,hr2-93,2,HR


In [5]:
len(votes_and_parties_df)

8777306

In [50]:
test_set = votes_and_parties_df[votes_and_parties_df[
    'bioguide_id'] == 'A000216'].reset_index(drop=True)  

In [28]:
test_set['majority_party'].value_counts()

D    423
R    310
B    134
Name: majority_party, dtype: int64

In [9]:
def ideology_compiler(vote_and_party_df):
    legi_count = 0 
    df_list = [] 
    for legi_id in vote_and_party_df['bioguide_id'].unique():
        member_id = legi_id
        member_votes = vote_and_party_df[
            vote_and_party_df['bioguide_id'] == legi_id].reset_index(drop=True)
        
        party = list(member_votes['party'])
        bioguide_id = list(member_votes['bioguide_id'])
        thomas_id = list(member_votes['thomas_id'])
        congress_numbers = list(member_votes['congress'])
        bill_ids = list(member_votes['bill_id'])
        vote_ids = list(member_votes['vote_id'])
        member_vote = list(member_votes['vote'])
        bill_labels = list(zip(member_votes['major_label 1'],
                            member_votes['major_label 2'],
                            member_votes['major_label 3']))
        bill_party_positions = list(member_votes['majority_party'])
        topic_1 = []
        topic_2 = []
        topic_3 = []
        topic_4 = []
        topic_5 = []
        topic_6 = []
        topic_7 = []
        topic_8 = []
        topic_9 = []
        topic_10 = []
        topic_12 = []
        topic_13 = []
        topic_14 = []
        topic_15 = []
        topic_16 = []
        topic_17 = []
        topic_18 = []
        topic_19 = []
        topic_20 = []
        topic_21 = []
        topic_99 = []
        out_dict = {1: {'ideology': topic_1, 'count': 0, 'key': 1},
                    2: {'ideology': topic_2, 'count':0, 'key': 2},
                    3: {'ideology': topic_3, 'count':0, 'key': 3},
                    4: {'ideology': topic_4, 'count':0, 'key': 4},
                    5: {'ideology': topic_5, 'count':0, 'key': 5},
                    6: {'ideology': topic_6, 'count':0, 'key': 6},
                    7: {'ideology': topic_7, 'count':0, 'key': 7},
                    8: {'ideology': topic_8, 'count':0, 'key': 8},
                    9: {'ideology': topic_9, 'count':0, 'key': 9},
                    10: {'ideology': topic_10, 'count':0, 'key': 10},
                    12: {'ideology': topic_12, 'count':0, 'key': 12},
                    13: {'ideology': topic_13, 'count':0, 'key': 13},
                    14: {'ideology': topic_14, 'count':0, 'key': 14},
                    15: {'ideology': topic_15, 'count':0, 'key': 15},
                    16: {'ideology': topic_16, 'count':0, 'key': 16},
                    17: {'ideology': topic_17, 'count':0, 'key': 17},
                    18: {'ideology': topic_18, 'count':0, 'key': 18},
                    19: {'ideology': topic_19, 'count':0, 'key': 19},
                    20: {'ideology': topic_20, 'count':0, 'key': 20},
                    21: {'ideology': topic_21, 'count':0, 'key': 21},
                    99: {'ideology': topic_99, 'count':0, 'key': 99}}

        for vote in range(len(vote_ids)):
            member_position = None
            if bill_party_positions[vote] == 'R':
                member_position = member_vote[vote]
            elif bill_party_positions[vote] == 'B':
                member_position = 21
            else:
                member_position = member_vote[vote] * -1 
            if member_position == 21:
                for ideology in out_dict:
                    if out_dict[ideology]['count'] == 0:
                        out_dict[ideology]['ideology'].append(0)
                    else:
                        out_dict[ideology]['ideology'].append(
                            out_dict[ideology]['ideology'][-1])
            else:
                topics = []
                topics.append(int(bill_labels[vote][0]))
                topics.append(int(bill_labels[vote][1]))
                topics.append(int(bill_labels[vote][2]))
                non_topics = [] 
                for bill_topic in out_dict:
                    if out_dict[bill_topic]['key'] not in topics:
                        non_topics.append(out_dict[bill_topic]['key'])
                    else:
                        pass 
                for topic in topics:
                    if out_dict[topic]['count'] == 0:
                        out_dict[topic]['ideology'].append(
                            member_position)
                        out_dict[topic]['count'] +=1
                    else:
                        out_dict[topic]['count'] +=1
                        out_dict[topic]['ideology'].append(
                            float(out_dict[topic]['ideology'][-1]) + (
                                member_position / out_dict[topic]['count']))  
                for non_topic in non_topics:
                        if out_dict[non_topic]['count'] == 0:
                            out_dict[non_topic]['ideology'].append(0)
                        else:
                            out_dict[non_topic]['ideology'].append(
                                out_dict[non_topic]['ideology'][-1])
                                     
        data_dict = {'bill_id': bill_ids, 'vote_id': vote_ids, 'party': party,
                     'majority support party': bill_party_positions,
                     'congress': congress_numbers, 'bioguide_id': bioguide_id,
                     'thomas_id': thomas_id, 'topic_1': topic_1,'topic_2': topic_2,
                     'topic_3': topic_3, 'topic_4': topic_4, 'topic_5': topic_5,
                     'topic_6': topic_6, 'topic_7': topic_7, 'topic_8': topic_8,
                     'topic_9': topic_9, 'topic_10': topic_10, 'topic_12': topic_12,
                     'topic_13': topic_13, 'topic_14': topic_14, 'topic_15': topic_15,
                     'topic_16': topic_16, 'topic_17': topic_17, 'topic_18': topic_18,
                     'topic_19': topic_19, 'topic_20': topic_20, 'topic_21': topic_21,
                     'topic_99': topic_99}
        
        output_df = pd.DataFrame(data_dict)
        df_list.append(output_df) 
        legi_count +=1
        if legi_count < vote_and_party_df['bioguide_id'].nunique():
            print ((legi_count/vote_and_party_df[
                'bioguide_id'].nunique())*100, '% Complete...')
        else:
            print ('100% Complete. Saving output to .csv file...')
    final_output = pd.concat(df_list, ignore_index=True)
    final_output.to_csv('all_member_ideologies.csv', index=False) 
    
    return final_output                                

In [7]:
all_ideologies = ideology_compiler(votes_and_parties_df) 

0.045934772622875514 % Complete...
0.09186954524575103 % Complete...
0.13780431786862654 % Complete...
0.18373909049150206 % Complete...
0.22967386311437757 % Complete...
0.2756086357372531 % Complete...
0.3215434083601286 % Complete...
0.3674781809830041 % Complete...
0.41341295360587965 % Complete...
0.45934772622875514 % Complete...
0.5052824988516307 % Complete...
0.5512172714745062 % Complete...
0.5971520440973818 % Complete...
0.6430868167202572 % Complete...
0.6890215893431328 % Complete...
0.7349563619660082 % Complete...
0.7808911345888837 % Complete...
0.8268259072117593 % Complete...
0.8727606798346348 % Complete...
0.9186954524575103 % Complete...
0.964630225080386 % Complete...
1.0105649977032614 % Complete...
1.056499770326137 % Complete...
1.1024345429490123 % Complete...
1.148369315571888 % Complete...
1.1943040881947635 % Complete...
1.240238860817639 % Complete...
1.2861736334405145 % Complete...
1.3321084060633899 % Complete...
1.3780431786862657 % Complete...
1.4239

11.713367018833257 % Complete...
11.759301791456132 % Complete...
11.805236564079008 % Complete...
11.851171336701883 % Complete...
11.89710610932476 % Complete...
11.943040881947635 % Complete...
11.98897565457051 % Complete...
12.034910427193385 % Complete...
12.080845199816261 % Complete...
12.126779972439136 % Complete...
12.172714745062011 % Complete...
12.218649517684888 % Complete...
12.264584290307763 % Complete...
12.31051906293064 % Complete...
12.356453835553515 % Complete...
12.40238860817639 % Complete...
12.448323380799264 % Complete...
12.49425815342214 % Complete...
12.540192926045016 % Complete...
12.586127698667893 % Complete...
12.632062471290768 % Complete...
12.67799724391364 % Complete...
12.723932016536518 % Complete...
12.769866789159392 % Complete...
12.81580156178227 % Complete...
12.861736334405144 % Complete...
12.90767110702802 % Complete...
12.953605879650896 % Complete...
12.999540652273772 % Complete...
13.045475424896647 % Complete...
13.091410197519524

23.24299494717501 % Complete...
23.28892971979789 % Complete...
23.334864492420763 % Complete...
23.38079926504364 % Complete...
23.426734037666513 % Complete...
23.47266881028939 % Complete...
23.518603582912263 % Complete...
23.564538355535138 % Complete...
23.610473128158016 % Complete...
23.65640790078089 % Complete...
23.702342673403766 % Complete...
23.74827744602664 % Complete...
23.79421221864952 % Complete...
23.84014699127239 % Complete...
23.88608176389527 % Complete...
23.932016536518145 % Complete...
23.97795130914102 % Complete...
24.023886081763894 % Complete...
24.06982085438677 % Complete...
24.115755627009648 % Complete...
24.161690399632523 % Complete...
24.207625172255398 % Complete...
24.253559944878273 % Complete...
24.29949471750115 % Complete...
24.345429490124022 % Complete...
24.3913642627469 % Complete...
24.437299035369776 % Complete...
24.48323380799265 % Complete...
24.529168580615526 % Complete...
24.5751033532384 % Complete...
24.62103812586128 % Complet

34.86449242076252 % Complete...
34.91042719338539 % Complete...
34.956361966008274 % Complete...
35.002296738631145 % Complete...
35.048231511254016 % Complete...
35.094166283876895 % Complete...
35.14010105649977 % Complete...
35.186035829122645 % Complete...
35.23197060174552 % Complete...
35.277905374368395 % Complete...
35.32384014699127 % Complete...
35.36977491961415 % Complete...
35.41570969223702 % Complete...
35.4616444648599 % Complete...
35.50757923748277 % Complete...
35.55351401010565 % Complete...
35.59944878272853 % Complete...
35.6453835553514 % Complete...
35.69131832797427 % Complete...
35.73725310059715 % Complete...
35.78318787322003 % Complete...
35.8291226458429 % Complete...
35.87505741846578 % Complete...
35.92099219108865 % Complete...
35.96692696371153 % Complete...
36.01286173633441 % Complete...
36.05879650895728 % Complete...
36.10473128158016 % Complete...
36.15066605420303 % Complete...
36.19660082682591 % Complete...
36.242535599448786 % Complete...
36.2

46.57785943959578 % Complete...
46.62379421221865 % Complete...
46.66972898484153 % Complete...
46.7156637574644 % Complete...
46.76159853008728 % Complete...
46.807533302710155 % Complete...
46.853468075333026 % Complete...
46.8994028479559 % Complete...
46.94533762057878 % Complete...
46.991272393201655 % Complete...
47.037207165824526 % Complete...
47.083141938447405 % Complete...
47.129076711070276 % Complete...
47.175011483693154 % Complete...
47.22094625631603 % Complete...
47.266881028938904 % Complete...
47.31281580156178 % Complete...
47.35875057418466 % Complete...
47.40468534680753 % Complete...
47.45062011943041 % Complete...
47.49655489205328 % Complete...
47.542489664676154 % Complete...
47.58842443729904 % Complete...
47.63435920992191 % Complete...
47.68029398254478 % Complete...
47.72622875516766 % Complete...
47.77216352779054 % Complete...
47.81809830041341 % Complete...
47.86403307303629 % Complete...
47.90996784565916 % Complete...
47.95590261828204 % Complete...
4

58.29122645842904 % Complete...
58.3371612310519 % Complete...
58.38309600367479 % Complete...
58.42903077629765 % Complete...
58.47496554892053 % Complete...
58.520900321543415 % Complete...
58.56683509416628 % Complete...
58.61276986678916 % Complete...
58.65870463941204 % Complete...
58.70463941203491 % Complete...
58.750574184657786 % Complete...
58.79650895728066 % Complete...
58.842443729903536 % Complete...
58.888378502526415 % Complete...
58.934313275149286 % Complete...
58.980248047772164 % Complete...
59.026182820395036 % Complete...
59.072117593017914 % Complete...
59.11805236564079 % Complete...
59.163987138263664 % Complete...
59.20992191088654 % Complete...
59.255856683509414 % Complete...
59.30179145613229 % Complete...
59.34772622875517 % Complete...
59.39366100137804 % Complete...
59.43959577400092 % Complete...
59.4855305466238 % Complete...
59.53146531924667 % Complete...
59.57740009186955 % Complete...
59.62333486449241 % Complete...
59.6692696371153 % Complete...
5

70.05052824988516 % Complete...
70.09646302250803 % Complete...
70.14239779513092 % Complete...
70.18833256775379 % Complete...
70.23426734037666 % Complete...
70.28020211299955 % Complete...
70.32613688562242 % Complete...
70.37207165824529 % Complete...
70.41800643086816 % Complete...
70.46394120349105 % Complete...
70.50987597611392 % Complete...
70.55581074873679 % Complete...
70.60174552135967 % Complete...
70.64768029398255 % Complete...
70.69361506660542 % Complete...
70.7395498392283 % Complete...
70.78548461185117 % Complete...
70.83141938447405 % Complete...
70.87735415709693 % Complete...
70.9232889297198 % Complete...
70.96922370234267 % Complete...
71.01515847496555 % Complete...
71.06109324758843 % Complete...
71.1070280202113 % Complete...
71.15296279283417 % Complete...
71.19889756545706 % Complete...
71.24483233807992 % Complete...
71.2907671107028 % Complete...
71.33670188332569 % Complete...
71.38263665594855 % Complete...
71.42857142857143 % Complete...
71.474506201

81.90169958658706 % Complete...
81.94763435920991 % Complete...
81.9935691318328 % Complete...
82.03950390445567 % Complete...
82.08543867707854 % Complete...
82.13137344970143 % Complete...
82.1773082223243 % Complete...
82.22324299494717 % Complete...
82.26917776757004 % Complete...
82.31511254019293 % Complete...
82.3610473128158 % Complete...
82.40698208543867 % Complete...
82.45291685806156 % Complete...
82.49885163068443 % Complete...
82.5447864033073 % Complete...
82.59072117593018 % Complete...
82.63665594855306 % Complete...
82.68259072117593 % Complete...
82.72852549379881 % Complete...
82.77446026642168 % Complete...
82.82039503904456 % Complete...
82.86632981166743 % Complete...
82.91226458429031 % Complete...
82.95819935691318 % Complete...
83.00413412953606 % Complete...
83.05006890215894 % Complete...
83.09600367478181 % Complete...
83.14193844740468 % Complete...
83.18787322002757 % Complete...
83.23380799265043 % Complete...
83.27974276527331 % Complete...
83.325677537

93.75287092328894 % Complete...
93.7988056959118 % Complete...
93.84474046853468 % Complete...
93.89067524115757 % Complete...
93.93661001378042 % Complete...
93.98254478640331 % Complete...
94.02847955902618 % Complete...
94.07441433164905 % Complete...
94.12034910427194 % Complete...
94.16628387689481 % Complete...
94.21221864951768 % Complete...
94.25815342214055 % Complete...
94.30408819476344 % Complete...
94.35002296738631 % Complete...
94.39595774000918 % Complete...
94.44189251263207 % Complete...
94.48782728525494 % Complete...
94.53376205787781 % Complete...
94.5796968305007 % Complete...
94.62563160312357 % Complete...
94.67156637574644 % Complete...
94.71750114836932 % Complete...
94.7634359209922 % Complete...
94.80937069361507 % Complete...
94.85530546623794 % Complete...
94.90124023886082 % Complete...
94.9471750114837 % Complete...
94.99310978410657 % Complete...
95.03904455672945 % Complete...
95.08497932935231 % Complete...
95.1309141019752 % Complete...
95.1768488745

In [2]:
ideologies =  pd.read_csv('all_member_ideologies.csv')

In [3]:
test_set = ideologies[ideologies['bioguide_id'] == 'A000216'].reset_index(drop=True) 

In [8]:
bills['major_label 1'].fillna(404, inplace=True)
bills['major_label 2'].fillna(404, inplace=True)
bills['major_label 3'].fillna(404, inplace=True)
bills['major_label 1'] = bills['major_label 1'].astype(int)
bills['major_label 2'] = bills['major_label 2'].astype(int)
bills['major_label 3'] = bills['major_label 3'].astype(int) 

In [10]:
sorted_bills = bills.sort_values(['congress', 'bill_type', 'number'])
sorted_bills = sorted_bills.reset_index(drop=True)
sorted_bills = sorted_bills[['bill_id', 'major_label 1', 'major_label 2', 'major_label 3']]

In [ ]:
member_ideologies = split_ideologies_by_member(ideologies) 

In [16]:
def split_ideologies_by_member(ideology_df):
    for bioguide in ideology_df['bioguide_id'].unique():
        member_df = ideology_df[ideology_df[
            'bioguide_id'] == bioguide].reset_index(drop=True)
        member_df.to_csv(('data_store/member_ideology/'+str(
            bioguide) + '_ideology_df.csv'), index=False)
    
        

In [13]:
test = split_ideologies_by_member(test_set)

In [14]:
test 

,bill_id,vote_id,party,majority support party,congress,bioguide_id,thomas_id,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_99
0,hr2-93,hr2-93-575,Republican,R,93.0,A000216,27.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0
1,hr2-93,hr2-93-576,Republican,R,93.0,A000216,27.0,0.000000,0.000000,0.000000,0.000000,1.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.500000,0.000000,0.000000,0.000000,0.000000,1.500000,0.000000,0.0
2,hr2-93,hr2-93-577,Republican,D,93.0,A000216,27.0,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.0
3,hr2-93,hr2-93-578,Republican,B,93.0,A000216,27.0,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.0
4,hr2-93,hr2-93-905,Republican,B,93.0,A000216,27.0,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.0
5,hr17-93,hr17-93-25,Republican,R,93.0,A000216,27.0,0.000000,0.000000,1.000000,0.000000,2.083333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,1.833333,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.0
6,hr17-93,hr17-93-26,Republican,D,93.0,A000216,27.0,0.000000,0.000000,1.500000,0.000000,2.283333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.500000,0.000000,1.833333,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.0
7,hr29-93,hr29-93-79,Republican,B,93.0,A000216,27.0,0.000000,0.000000,1.500000,0.000000,2.283333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.500000,0.000000,1.833333,0.000000,0.000000,0.000000,0.000000,1.833333,0.000000,0.0
8,hr37-93,hr37-93-339,Republican,D,93.0,A000216,27.0,0.000000,0.000000,1.500000,-1.000000,2.283333,0.000000,-1.000000,0.000000,0.0,0.000000,0.000000,1.500000,0.000000,1.833333,0.000000,0.000000,0.000000,-1.000000,1.833333,0.000000,0.0
9,hr69-93,hr69-93-611,Republican,D,93.0,A000216,27.0,0.000000,1.000000,1.500000,-1.000000,2.283333,1.000000,-1.000000,0.000000,0.0,0.000000,0.000000,1.833333,0.000000,1.833333,0.000000,0.000000,0.000000,-1.000000,1.833333,0.000000,0.0


In [22]:
def compile_ideology_by_congress(vote_and_party_df):
    df_list = [] 
    for legi_id in vote_and_party_df['bioguide_id'].unique():
        member_df = pd.read_csv('data_store/member_ideology/'+str(
            legi_id)+'_ideology_df.csv')
        df_list.append(member_df)
    total_df = pd.concat(df_list)
    for congress in total_df['congress'].unique():
        congress_df = total_df[total_df['congress'] == congress].reset_index(drop=True)
        congress_df.to_csv('data_store/ideology_by_congress/congress_'+str(
            congress)+'_ideology_df.csv', index=False)

In [23]:
all_ideologies_by_congress = compile_ideology_by_congress(votes_and_parties_df) 

If I remember correctly, the cosponsor ID format changes in 2012, so I should be sure that this method of assessing ideology is compatible with that ID format. I believe it should be fine as I am linking instances from the member dataframe which would include the other ID format, but it wouldn't hurt to look back at the metadata compiler just to make sure. 